In [3]:
%matplotlib inline

from matplotlib import pyplot as plt
from matplotlib import patches as mpatches
import numpy as np
import pandas as pd
import math
import sys

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [4]:
def plot_projections(ts, points, method_name):
    fig, ax = plt.subplots(figsize=(40, 40))
    ax.set_xlim([ts.min(), ts.max()])
    ax.set_ylim([points.min(), points.max()])
    
    red_patch = mpatches.Patch(color='red', label='X trajectory')
    green_patch = mpatches.Patch(color='green', label='Y trajectory ')
    blue_patch = mpatches.Patch(color='blue', label='Z trajectory')
    
    plt.legend(handles=[red_patch, green_patch, blue_patch], prop={'size':30}, loc=2)
    plt.title(method_name, size=30)
    ax.scatter(ts, points[:, 0], color='red', marker=',')
    ax.scatter(ts, points[:, 1], color='green', marker=',')
    ax.scatter(ts, points[:, 2], color='blue', marker=',')
    
def plot3D(points, method_name):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(points[:,0],points[:,1],points[:,2])

In [16]:
sigma = 10
b = 8.0/3
r=24
#r in [0..30]
#dx/dt=-sigma*x+sigma*y
#dy/dt=-x*z+r*x-y
#dz/dt=x*y-b*z
def f(vec): 
    x,y,z=vec
    return np.array([-sigma*x+sigma*y, -x*z+r*x-y, x*y-b*z])
start_x=np.array([10.0, -7.0, 4.0])
start_x

array([ 10.,  -7.,   4.])

In [28]:
def converge_to(method, start_x):
    ts, points = method(start_x)
    return points[-1]

In [17]:
def euler_method(start_x, t_a=0, t_b=100, dt=0.001):
    cur_t = t_a
    cur_x = np.copy(start_x)
    ts = [cur_t]
    way = [cur_x]
    while cur_t <= t_b:
        cur_x += f(cur_x)*dt
        cur_t += dt
        way.append(np.copy(cur_x))
        ts.append(cur_t)
    return (np.array(ts), np.array(way))

def runge_kutta_method(start_x, t_a=0, t_b=100, dt=0.001):
    cur_t = t_a
    cur_x = np.copy(start_x)
    ts = [cur_t]
    way = [cur_x]
    while cur_t <= t_b:
        k1 = f(cur_x)
        k2 = f(cur_x+dt*0.5*k1)
        k3 = f(cur_x+dt*0.5*k2)
        k4 = f(cur_x+dt*k3)
        cur_x = cur_x+dt/6*(k1+2*k2+2*k3+k4)
        cur_t += dt
        way.append(np.copy(cur_x))
        ts.append(cur_t)
    return (np.array(ts), np.array(way))


In [18]:
EPS = 1e-9

def solve_cubic_equation(u, y0): # solves y=u(y) using Newton
    h = np.polysub(u, np.array([1, 0]))
    dh = np.polyder(h)
    
    cur_y = y0
    next_y = cur_y - np.polyval(h, cur_y) / np.polyval(dh, cur_y)
    while (np.abs(np.polyval(h, cur_y)) > EPS):
        cur_y = np.copy(next_y)
        next_y = cur_y - np.polyval(h, cur_y) / np.polyval(dh, cur_y)
    return cur_y

def solve_system_equality(cur_x, dt):
    x0, y0, z0 = cur_x
    ax = np.array([dt*sigma, x0]) / (1+dt*sigma)
     
    az = np.polyadd(np.polymul(ax, np.array([1, 0]))*dt, 
                    np.array([z0])) / (1+dt*b)    
    
    # y=y0-dt*ax*az+dt*r*ax
    fixy = np.array([0,y0])
    fixy = np.polysub(fixy, dt*np.polymul(ax,az))
    fixy = np.polyadd(fixy, dt*r*ax)
    fixy = np.polysub(fixy, np.array([dt, 0]))
    
    ny = solve_cubic_equation(fixy, y0)
    nx = np.polyval(ax, ny)
    nz = np.polyval(az, ny)
    result = np.array([nx, ny, nz])
    return result

def implicit_euler_method(start_x, t_a=0, t_b=20, dt=0.001):
    cur_t = t_a
    cur_x = np.copy(start_x)
    ts = [cur_t]
    way = [cur_x]
    while cur_t <= t_b:
        cur_x = solve_system_equality(cur_x, dt)
        cur_t += dt
        way.append(np.copy(cur_x))
        ts.append(cur_t)
    return (np.array(ts), np.array(way))

In [23]:
def adams_method(start_x, t_a=0, t_b=100, dt=0.001):
    cur_t = t_a
    cur_x = np.copy(start_x)
    ts = [cur_t]
    way = [cur_x]
    while cur_t <= t_b:
        if len(way) <= 4:
            cur_x += f(cur_x)*dt
            cur_t += dt
        else:
            p0, p1, p2, p3 = map(f, way[-4:])
            pred = cur_x + (dt/24) * (55*p0-59*p1+37*p2-9*p3)
            p_1 = f(pred)
            corr = cur_x + (dt/24) * (9*p_1+19*p0-5*p1+p2)
            cur_x = np.copy(corr)
            cur_t += dt
        way.append(np.copy(cur_x))
        ts.append(cur_t)
            
    return (np.array(ts), np.array(way))

In [36]:
def final_points(method, start_x):
    global r
    for r_id in range(100):
        r = 30.0 * r_id / 100
        x, y, z = converge_to(method, start_x)
        print("%.2f converjed to (%.3f %3.f %3.f)" % (r, x, y, z))

In [ ]:
final_points(euler_method, start_x)

In [ ]:
plot_projections(ts, points, "Euler method")

In [ ]:
plot3D(points,'Adams method')

In [15]:
np.savetxt("data.csv", points, delimiter=' ')

In [11]:
np.polysub(np.array([1,0]), np.array([0,2,3]))

array([ 0, -1, -3])

In [98]:
ts, points = implicit_euler_method(start_x)

In [92]:
cur_x=np.array([1,0,0])
solve_system_equality(cur_x, 1.0)

[ -0.22539444  -0.04507889  22.63410969   2.36363636]
[-0.00840345 -0.10924379  0.00025037]
[-0.00840345 -0.10924379  0.00025037]


array([-0.00840345, -0.10924379,  0.00025037])

In [ ]:
x=(10/11)*y
z=(30/121)*y^2
xz=